In [1]:
import kagglehub
from pathlib import Path

path = kagglehub.dataset_download("deepshah16/song-lyrics-dataset")
p = Path(path)
print("Path:", p)
print("Files:", list(p.rglob("*"))[:20])


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path: /home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5
Files: [PosixPath('/home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/csv'), PosixPath('/home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/json files'), PosixPath('/home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/csv/PostMalone.csv'), PosixPath('/home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/csv/KatyPerry.csv'), PosixPath('/home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/csv/BillieEilish.csv'), PosixPath('/home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/csv/Khalid.csv'), PosixPath('/home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/csv/Drake.csv'), PosixPath('/home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/csv/ArianaGrande.cs

In [2]:
import pandas as pd
from pathlib import Path

# 1) gdje je dataset
dataset_dir = Path(path)

# 2) nađi csv (ako ih ima više, uzmi pravi)
csv_file = next(dataset_dir.rglob("*.csv"))
print("Using:", csv_file)

df = pd.read_csv(csv_file)

# 3) provjeri stupce da znamo koji sadrži tekst
print(df.columns)

# 4) PROMIJENI OVO ako ti se stupac drugačije zove:
TEXT_COL = "lyrics"   # često je "lyrics" ili "text"
TITLE_COL = "title"   # opcionalno

out_dir = Path("data/raw")
out_dir.mkdir(parents=True, exist_ok=True)

for i, row in df.iterrows():
    lyrics = str(row.get(TEXT_COL, "")).strip()
    if not lyrics or lyrics.lower() == "nan":
        continue
    title = str(row.get(TITLE_COL, f"song_{i}")).strip().replace("/", "_")
    (out_dir / f"{i:06d}_{title}.txt").write_text(lyrics, encoding="utf-8", errors="ignore")

print("Done. Wrote .txt files to:", out_dir)



Using: /home/codespace/.cache/kagglehub/datasets/deepshah16/song-lyrics-dataset/versions/5/csv/PostMalone.csv
Index(['Unnamed: 0', 'Artist', 'Title', 'Album', 'Year', 'Date', 'Lyric'], dtype='object')
Done. Wrote .txt files to: data/raw


In [3]:
import json
from pathlib import Path

src_dir = Path("json files")      # folder gdje su Lyrics_*.json
out_dir = Path("data/raw")
out_dir.mkdir(parents=True, exist_ok=True)

all_lyrics = []
for fp in src_dir.glob("*.json"):
    obj = json.loads(fp.read_text(encoding="utf-8", errors="ignore"))

    # pokušaj izvući tekstove bez znanja točne strukture
    def collect(o):
        if isinstance(o, dict):
            for k, v in o.items():
                if isinstance(v, str) and len(v) > 30 and ("[" in v or "\n" in v or "chorus" in v.lower()):
                    all_lyrics.append(v)
                else:
                    collect(v)
        elif isinstance(o, list):
            for x in o:
                collect(x)

    collect(obj)

# očisti i spoji u jedan korpus
all_lyrics = [t.strip() for t in all_lyrics if t and t.strip()]
(out_dir / "corpus.txt").write_text("\n\n".join(all_lyrics), encoding="utf-8")

print("OK. Broj tekstova:", len(all_lyrics))
print("Napisano:", out_dir / "corpus.txt")


OK. Broj tekstova: 11774
Napisano: data/raw/corpus.txt
